In [1]:
import numpy as np
import xarray as xr
import pandas as pd 
import sys 
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1, 
                    exclude_age1=True)

A check of the initialization file needs to be done!
A check of the survey year file needs to be done!
Loading biological data ...


In [2]:
# epro_2019.strata_df["length_average_haul"] = np.nan
# epro_2019.strata_df["TS_lin_haul"] = np.nan
# epro_2019.strata_df["sig_bs_haul"] = np.nan
# for haul_num in epro_2019.specimen_df.index.unique():
    
#     all_len = epro_2019.specimen_df.loc[haul_num]['Length'] 
    
#     # TODO: replace with length_ds?
#     if haul_num in epro_2019.length_df.index:
#         all_len = np.concatenate([all_len, 
#                                   epro_2019.length_df.loc[haul_num]['Length']])
    
    
#     TS0j = 20.0*np.log10(all_len) - 68.0
#     TSj = 10.0*np.log10(np.nanmean(10.0**(TS0j/10.0)))
#     epro_2019.strata_df.loc[haul_num, "TS_lin_haul"] = TSj
#     epro_2019.strata_df.loc[haul_num, "sig_bs_haul"] = 10.0**(TSj/10.0)
#     epro_2019.strata_df.loc[haul_num, "length_average_haul"] = np.nanmean(all_len)

# epro_2019.strata_df["length_average_haul"] = np.nan
# epro_2019.strata_df["TS_lin_haul"] = np.nan
# epro_2019.strata_df["sig_bs_haul"] = np.nan
# for haul_num in epro_2019.specimen_df.index.unique():
    
#     all_len = epro_2019.specimen_df.loc[haul_num]['Length'] 
    
#     # TODO: replace with length_ds?
#     if haul_num in epro_2019.length_df.index:
#         all_len = np.concatenate([all_len, 
#                                   epro_2019.length_df.loc[haul_num]['Length']])
    
    
#     TS0j = 20.0*np.log10(all_len) - 68.0
#     TSj = 10.0*np.log10(np.nanmean(10.0**(TS0j/10.0)))
#     epro_2019.strata_df.loc[haul_num, "TS_lin_haul"] = TSj
#     epro_2019.strata_df.loc[haul_num, "sig_bs_haul"] = 10.0**(TSj/10.0)
#     epro_2019.strata_df.loc[haul_num, "length_average_haul"] = np.nanmean(all_len)

    
# epro_2019.strata_ds = epro_2019.strata_df.to_xarray()


# epro_2019.strata_ds["sig_bs"] = epro_2019.strata_ds.sig_bs_haul.mean(dim="Haul", skipna=True)
# epro_2019.strata_ds["sig_b"] = 4.0*np.pi*epro_2019.strata_ds["sig_bs"]



epro_2019.strata_ds


<xarray.Dataset>
Dimensions:              (Haul: 75, strata: 9)
Coordinates:
  * Haul                 (Haul) int64 1 3 7 8 9 10 ... 211 213 214 216 218 222
  * strata               (strata) int64 0 1 2 3 4 5 6 7 8
Data variables:
    Year                 (Haul, strata) float64 nan 2.019e+03 nan ... nan nan
    wt                   (Haul, strata) float64 nan 1.0 nan nan ... nan nan nan
    length_average_haul  (Haul, strata) float64 nan 23.32 nan ... nan nan nan
    TS_lin_haul          (Haul, strata) float64 nan -40.61 nan ... nan nan nan
    sig_bs_haul          (Haul, strata) float64 nan 8.699e-05 nan ... nan nan
    sig_bs               (strata) float64 0.000425 8.4e-05 ... 0.0004805
    sig_b                (strata) float64 0.005341 0.001056 ... 0.006039

In [103]:
strata_num = 1
4.0*np.pi*np.nanmean(epro_2019.strata_df.loc[(slice(None), strata_num), "sig_bs"])

0.0010556072981591567

In [106]:
epro_2019.strata_df.to_xarray()

<xarray.Dataset>
Dimensions:         (Haul: 75, strata: 9)
Coordinates:
  * Haul            (Haul) int64 1 3 7 8 9 10 11 ... 208 211 213 214 216 218 222
  * strata          (strata) int64 0 1 2 3 4 5 6 7 8
Data variables:
    Year            (Haul, strata) float64 nan 2.019e+03 nan nan ... nan nan nan
    wt              (Haul, strata) float64 nan 1.0 nan nan ... nan nan nan nan
    length_average  (Haul, strata) float64 nan 23.32 nan nan ... nan nan nan nan
    TS_lin          (Haul, strata) float64 nan -40.61 nan nan ... nan nan nan
    sig_bs          (Haul, strata) float64 nan 8.699e-05 nan nan ... nan nan nan

In [81]:
# transect_reduction_fraction = 0.0 
# stratification_index = 1
# KS_stratification = 1
# epro_2019.load_strata.get_strata_data(stratification_index, KS_stratification, transect_reduction_fraction)